In this notebook we're going to prepare the feature and target vectors for two scores, and then we'll do the grouping lists that we are gonna use for cross validation.

In [1]:
import nilearn
import pandas as pd
import numpy as np
import glob as gl
import os
%matplotlib inline

from nilearn.plotting import plot_connectome
from nilearn.connectome import ConnectivityMeasure

c:\users\ayman\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
c:\users\ayman\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
c:\users\ayman\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


Let's start with dataframes.

In [2]:
#Set of scores:
Gold_data = pd.read_table('phenotype/GoldMSI.tsv').dropna()

#lists of retained subjects:

subjects_list =[i[-3:] for i in list(Gold_data['participant_id'])]

To create the groups lists we'll need the following function that returns the session number given the filename and the two dictionnaries that bind subject numbers to their target values.

In [3]:
list(Gold_data.iloc[0,1:])

[19.0, 7.0]

In [4]:
def Session_from_name(name):
    param1 = name[90:92]
    param2 = name[97:99]
    if param1 == 'AP' and param2 == '01':
        return '1'
    elif param1 == 'AP' and param2 == '02':
        return '2'
    elif param1 == 'PA' and param2 == '01':
        return '3'
    else:
        return '4'

In [8]:
Y_dict ={}

for i in range(len(subjects_list)):
    Y_dict[subjects_list[i]] = list(Gold_data.iloc[i,1:])
    

Now we'll have to scoure the fMRI dataset files to find those associated to subjects we're interested, we'll create the time series necessary for the tangent vectors' calculation, and at the same time we'll fill the groups and target lists.

In [9]:
Y=[]
group_bysubject=[]
time_series = []
for subject_num in subjects_list:
    for subject_file in gl.glob('allsubjs/subj_010'+subject_num+'*'):
        time_series.append(np.load(subject_file)['ts'])
        group_bysubject.append(subject_num)
        Y.append(Y_dict[subject_num])

The last step is creating the tangent vectors with the nilearn method ConnectivityMeasure.

In [10]:
tangent_measure = ConnectivityMeasure(kind='tangent',vectorize=True,discard_diagonal=True)
tangent_vectors = tangent_measure.fit_transform(time_series)

Now we'll save all the arrays for future usage/

In [25]:
np.savez('data.npz',X=tangent_vectors, Y=Y, groups_bysubject = group_bysubject)